In [10]:
%pip install llama-index-llms-google-genai llama-index
%pip install llama-index-utils-workflow

In [1]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')
tavily_key = userdata.get('tavily-key')

In [2]:
from llama_index.llms.google_genai import GoogleGenAI


llm = GoogleGenAI(
    model="models/gemini-2.0-flash-exp-image-generation",
    api_key=gemini_api_key,
)

resp = llm.complete("Who is Paul Graham?")
print(resp)

Paul Graham is a prominent figure in the tech world, best known for his work as a computer programmer, essayist, venture capitalist, and co-founder of the startup accelerator Y Combinator. Here's a breakdown of his key accomplishments and contributions:

*   **Computer Programmer and Author:** Graham holds a Ph.D. in computer science from Harvard University. He is known for his work on Lisp, a programming language, and for developing Viaweb, one of the first software-as-a-service (SaaS) companies, which was later acquired by Yahoo! and became Yahoo! Store. He's also the author of several influential books and essays on programming, startups, and technology.

*   **Essayist:** Graham is a prolific and influential essayist. His essays, published on his website, cover a wide range of topics, including startups, technology, design, writing, and philosophy. They are widely read and discussed in the tech community. Some of his most famous essays include "Hackers & Painters," "How to Start a 

In [3]:
%pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00


# Basics

In [9]:
from tavily import AsyncTavilyClient

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_key)
    return str(await client.search(query))

In [10]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

In [11]:
response = await agent.run(user_msg="What is the weather in San Francisco?")
print(str(response))

The weather in San Francisco on April 12, 2025 is partly cloudy with a temperature of 13.9°C (57.0°F). The wind is blowing from the WSW at 6.0 mph.


# Agent Worflow


In [3]:
!pip install llama-index-utils-workflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.2 MB/s eta 0:00:00


In [3]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context
)

In [13]:
class DBUtils():

  def __init__(self):
    pass

  def get_from_db_last_workout(self) -> int:
    return 2


In [25]:
from llama_index.core.workflow import Event

class BeginProcessingEvent(Event):
    bpe_output: str

class NotificationEvent(Event):
    ne_output: str

class LandmarksEvent(Event):
    le_output: str

class TrackingEvent(Event):
    te_output: str

class StopEvent(Event):
    result: str = ""


In [28]:
import re
import json
class WalkWithScottyWorkflow(Workflow):

    async def setup(self, ev: StartEvent) -> BeginProcessingEvent:
        print(ev.first_input)
        self.when_to_send_notification_agent = ev.when_to_send_notification_agent
        self.determine_distance_to_walk_agent = ev.determine_distance_to_walk_agent
        self.determine_route_agent = ev.determine_route_agent
        self.track_user_agent = ev.track_user_agent
        self.verify_user_milestones_agent = ev.verify_user_milestones_agent

        self.db_utils = DBUtils()
        last_workout = self.db_utils.get_from_db_last_workout()
        if last_workout >= 2:
          return BeginProcessingEvent(bpe_output="Setup complete.")
        else:
          return BeginProcessingEvent(bpe_output="Begin Processing Anyway. Setup complete.") #TODO: call the function that calls StartEvent at the very beginning maybe

    async def BeginProcessingStep(self, ev: BeginProcessingEvent) -> NotificationEvent | BeginProcessingEvent:
        result = self.when_to_send_notification_agent.chat(f"Determine if the current time is appropriate to send a workout reminder notification to the user. Just return 'Yes' or 'No'")
        if 'yes' in result.lower():
          return NotificationEvent(ne_output="tbd")
        else:
          return BeginProcessingEvent(bpe_output="tbd")

    async def CalculateGoalDistance(self, ctx: Context, ev: NotificationEvent) -> LandmarksEvent:
        result = self.determine_distance_to_walk_agent.chat("Can you please fetch the distance to watch based on the user's past walks. Return the distance in miles.")
        await ctx.set("distance_to_walk", result)
        return LandmarksEvent(le_output="tbd")

    async def DetermineRoute(self, ctx: Context, ev: LandmarksEvent) -> StopEvent:
        # should call the self.determine_route_agent to populate the following fields in the Context:
        #
        # - A json_object:
        #    - "current-location": fetch from GPS calling API, but hard code this to be Cohon University Centres long and lat, format arr[2] = [longitue, latitude]
        #    - "landmarks_to_visit": use Googlemaps api to get the long, lat of these three landmarks - Posner Hall - CMU, Westinghouse memorial, Phipps Conservatory
        #    - "landmarks_location" : array of three longitudes and latitudes
        #    - "estimated_total_time": populate the estimated total time by adding the time to go from current location, to all the three landmarks in the same sequence
        # The above json object should be returned by the determine_route_agent agent, and the prompt to that should ask it to generate it's result of calling all the tools in the above json

        # Get the walking distance from context
        distance_to_walk = await ctx.get("distance_to_walk")

        # Create the prompt for the route agent
        prompt = f"""
        I need you to plan a walking route for a user who wants to walk approximately {distance_to_walk} miles.

        Please create a JSON object with exactly these fields:
        1. "current-location": Use the get_current_location tool to get the coordinates of Cohon University Center
        2. "landmarks_to_visit": An array of these three landmarks in this order: ["Posner Hall - CMU", "Westinghouse memorial", "Phipps Conservatory"]
        3. "landmarks_location": An array containing the coordinates of each landmark in the same order. Use the get_landmark_location tool for each landmark.
        4. "estimated_total_time": Calculate the total walking time by:
          - Getting time from current location to first landmark
          - Adding time from first to second landmark
          - Adding time from second to third landmark
          Use the calculate_walking_time tool for each segment.

        Return ONLY the JSON object, with no additional explanation.
        """

        # Call the determine_route_agent
        result = self.determine_route_agent.chat(prompt)

        # Extract the JSON object from the response
        response_text = str(result)
        print(f"Route agent response received")

        try:
            # Find JSON content in the response
            json_match = re.search(r'({[\s\S]*})', response_text)
            if json_match:
                json_str = json_match.group(1)
                route_data = json.loads(json_str)

                # Store route data in context
                await ctx.set("route_data", route_data)

                # Create a nicely formatted route description for the output
                landmarks = route_data.get("landmarks_to_visit", [])
                locations = route_data.get("landmarks_location", [])
                total_time = route_data.get("estimated_total_time", 0)

                route_description = f"Walking route planned: "
                route_description += f"Starting from Cohon University Center, visiting {', '.join(landmarks)}. "
                route_description += f"Total estimated time: {total_time} minutes."

                return StopEvent()
            else:
                error_msg = "Could not extract JSON data from route agent's response"
                await ctx.set("route_error", error_msg)
                return StopEvent()

        except Exception as e:
            error_msg = f"Error processing route data: {str(e)}"
            await ctx.set("route_error", error_msg)
            return StopEvent()


w = WalkWithScottyWorkflow(timeout=10, verbose=True)
# result = await w.run(first_input="Start the workflow.")
# print(result)

from llama_index.utils.workflow import draw_all_possible_flows
draw_all_possible_flows(w, filename="basic_workflow.html")

WorkflowConfigurationError: At least one Event of type StopEvent must be returned by any step.

In [25]:
from llama_index.utils.workflow import draw_all_possible_flows
draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

basic_workflow.html


In [33]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg="Write me a report on the history of the web. Briefly describe the history of the world wide web, including the development of the internet and the development of the web, including 21st century developments"
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: ResearchAgent

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'history of the world wide web and internet development'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'history of the world wide web and internet development'}
  Output: name 'AsyncTavilyClient' is not defined
📤 Output: I was unable to retrieve information from the web. I will try a different query.

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'history of the world wide web'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'history of the world wide web'}
  Output: name 'AsyncTavilyClient' is not defined
📤 Output: I am still having trouble getting information from the web. I will try one more time, and if it doesn't work, I will have to hand off to another agent.

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': '21st century developments world wide web'}
🔧

# Final Backend

In [142]:
from typing import Dict, List, Any
from datetime import datetime
import random
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.google_genai import GoogleGenAI

# === TOOL FUNCTION DEFINITIONS ===

def check_calendar(**kwargs) -> str:
    """Stub for checking user's calendar availability."""
    busy_times = ["9:00 AM - 10:00 AM", "2:00 PM - 3:00 PM"]
    return f"User is busy during: {', '.join(busy_times)}"

def check_weather(**kwargs) -> str:
    """Stub for checking weather."""
    weather_conditions = ["Sunny", "Cloudy", "Raining", "Snowing", "Windy"]
    current_weather = random.choice(weather_conditions)
    return f"The current weather is {current_weather}."

def check_time_of_day(**kwargs) -> str:
    """Returns part of the day based on current system time."""
    hour = datetime.now().hour
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

def get_current_location(**kwargs) -> Dict[str, float]:
    """Returns a mock GPS location (CMU)."""
    return {"latitude": 40.444, "longitude": -79.945}  # CMU, Pittsburgh

def get_landmark_location(**kwargs) -> List[Dict[str, Any]]:
    """Returns mock locations of predefined landmarks."""
    return [
        {"name": "Posner Hall - CMU", "latitude": 40.442, "longitude": -79.950},
        {"name": "Westinghouse memorial", "latitude": 40.443, "longitude": -79.953},
        {"name": "Phipps Conservatory", "latitude": 40.439, "longitude": -79.946}
    ]

def get_estimated_time(**kwargs) -> str:
  return "20. minutes"

def calculate_walking_time(start: Dict[str, float] = None, end: Dict[str, float] = None, **kwargs) -> str:
    if not start or not end:
        return "Unknown time"

    lat_diff = abs(start["latitude"] - end["latitude"])
    lon_diff = abs(start["longitude"] - end["longitude"])
    distance = (lat_diff**2 + lon_diff**2)**0.5 * 111  # Rough km
    time_min = round((distance / 5) * 60)  # 5 km/h walking speed
    return f"{time_min} minutes"

def verify_image(**kwargs):
    return "valid"

def record_stats(**kwargs):
    return "Calories: 250, Steps: 4200"


# === AGENT SETUP ===

def create_agents(api_key):
    """Create all the agents needed for the WalkWithScotty workflow."""

    llm = GoogleGenAI(
        model="models/gemini-1.5-flash",
        api_key=api_key,
    )

    # Wrap tool functions
    calendar_tool = FunctionTool.from_defaults(
        fn=check_calendar,
        name="check_calendar",
        description="Checks user's calendar for availability."
    )

    estimated_time_tool = FunctionTool.from_defaults(
      fn=get_estimated_time,
      name="get_estimated_time",
      description="Returns a hardcoded estimate of the total walking time."
    )

    weather_tool = FunctionTool.from_defaults(
        fn=check_weather,
        name="check_weather",
        description="Retrieves current weather data."
    )

    time_tool = FunctionTool.from_defaults(
        fn=check_time_of_day,
        name="check_time_of_day",
        description="Determines whether it's morning, afternoon, or evening."
    )

    location_tool = FunctionTool.from_defaults(
        fn=get_current_location,
        name="get_current_location",
        description="Gets the user's current GPS location."
    )

    landmark_tool = FunctionTool.from_defaults(
        fn=get_landmark_location,
        name="get_landmark_location",
        description="Fetches landmark locations nearby."
    )

    walking_time_tool = FunctionTool.from_defaults(
        fn=calculate_walking_time,
        name="calculate_walking_time",
        description="Estimates the walking time between two GPS coordinates."
    )

    verify_image_tool = FunctionTool.from_defaults(
        fn=verify_image,
        name="verify_image",
        description="Verifies if the uploaded photo matches the location."
    )

    record_stats_tool = FunctionTool.from_defaults(
        fn=record_stats,
        name="record_stats",
        description="Updates the calories and steps after the walk."
    )


    # === AGENTS ===

    notification_agent = ReActAgent.from_tools(
        tools=[calendar_tool, weather_tool, time_tool],
        llm=llm,
        system_prompt=(
            "You are the NotificationAgent that determines if the current time is appropriate to send a workout reminder. "
            "You can check the user's calendar and current weather conditions make this decision. "
            "Respond with ONLY 'Yes' or 'No' based on your analysis."
        ),
        verbose=True
    )

    distance_agent = ReActAgent.from_tools(
        tools=[],
        llm=llm,
        system_prompt=(
            "You are the DistanceAgent that determines an appropriate walking distance for a user. "
            "When asked, return a distance in miles that would be appropriate for the user's next walk. "
            "If the user hasn't worked out in a few days (3 days or more), suggest a more moderate distance (1-2 miles). "
            "Be precise with your answer, including only the number in your response."
        ),
        verbose=True
    )

    route_agent = ReActAgent.from_tools(
        tools=[location_tool, landmark_tool, walking_time_tool, estimated_time_tool],
        llm=llm,
        system_prompt=(
            "You are the RouteAgent that plans walking routes with landmarks for a user's workout. "
            "You can get the user's current location, landmark locations, and calculate walking times. "
            "When asked to determine a route, create a JSON object with the current location, landmarks to visit, "
            "their locations, and the estimated total walking time."
        ),
        verbose=True
    )

    tracking_agent = ReActAgent.from_tools(
        tools=[],
        llm=llm,
        system_prompt=(
            "You are the TrackingAgent that tracks a user's progress during their walk. "
            "Respond with encouraging messages based on their progress."
        ),
        verbose=True
    )

    milestone_agent = ReActAgent.from_tools(
        tools=[],
        llm=llm,
        system_prompt=(
            "You are the VerifyMilestonesAgent that verifies a user has reached landmarks during their walk. "
            "Confirm when they've reached each landmark and provide encouraging feedback."
        ),
        verbose=True
    )

    return notification_agent, distance_agent, route_agent, tracking_agent, milestone_agent

In [143]:
class BeginProcessingEvent(Event):
    def __init__(self, bpe_output=""):
        super().__init__()
        self._bpe_output = bpe_output

    @property
    def bpe_output(self):
        return self._bpe_output

# Create custom event classes without using dataclasses
class BeginProcessingEvent(Event):
    def __init__(self, bpe_output=""):
        super().__init__()
        self._bpe_output = bpe_output

    @property
    def bpe_output(self):
        return self._bpe_output

class NotificationEvent(Event):
    def __init__(self, ne_output=""):
        super().__init__()
        self._ne_output = ne_output

    @property
    def ne_output(self):
        return self._ne_output

class LandmarksEvent(Event):
    def __init__(self, le_output=""):
        super().__init__()
        self._le_output = le_output

    @property
    def le_output(self):
        return self._le_output

class TrackingEvent(Event):
    def __init__(self, te_output=""):
        super().__init__()
        self._te_output = te_output

    @property
    def te_output(self):
        return self._te_output

class VerifyPhotoEvent(Event):
    def __init__(self, ve_output=""):
        super().__init__()
        self._ve_output = ve_output

    @property
    def ve_output(self):
        return self._ve_output

In [156]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
import re
import json

class MyWorkflow(Workflow):
    @step
    async def setup(self, ev: StartEvent) -> BeginProcessingEvent:
        print(ev.first_input)
        self.when_to_send_notification_agent = ev.when_to_send_notification_agent
        self.determine_distance_to_walk_agent = ev.determine_distance_to_walk_agent
        self.determine_route_agent = ev.determine_route_agent
        self.track_user_agent = ev.track_user_agent
        self.verify_user_milestones_agent = ev.verify_user_milestones_agent

        self.db_utils = DBUtils()
        last_workout = self.db_utils.get_from_db_last_workout()
        if last_workout >= 2:
            return BeginProcessingEvent(bpe_output="Setup complete.")
        else:
            return BeginProcessingEvent(bpe_output="Begin Processing. Setup complete.")

    @step
    async def BeginProcessing(self, ev: BeginProcessingEvent) -> NotificationEvent | BeginProcessingEvent:
        # Override agent behavior to always say Yes for testing
        return NotificationEvent(ne_output="Yes")

    @step
    async def CalculateGoalDistance(self, ctx: Context, ev: NotificationEvent) -> LandmarksEvent:
        result = self.determine_distance_to_walk_agent.chat("Can you please fetch the distance to walk. Return the distance in miles.")
        await ctx.set("distance_to_walk", result)
        return LandmarksEvent(le_output="tbd")

    @step
    async def DetermineRoute(self, ctx: Context, ev: LandmarksEvent) -> TrackingEvent:
        distance_to_walk = await ctx.get("distance_to_walk")

        # Call the route agent with a prompt to use tools for all fields except total time
        prompt = f"""
        You are a route planner agent. Use the available tools like `get_current_location` and `get_landmark_location`
        to generate a walking route for a user who wants to walk approximately {distance_to_walk} miles.

        Use the tools to:
        - Get the current GPS location of the user.
        - Get the coordinates of three landmarks: Posner Hall - CMU, Westinghouse memorial, and Phipps Conservatory.

        DO NOT calculate walking time using any tools. Instead, hardcode the estimated_total_time to 25.

        Return this JSON object only:
        {{
          "current-location": result_of_get_current_location,
          "landmarks_to_visit": ["Posner Hall - CMU", "Westinghouse memorial", "Phipps Conservatory"],
          "landmarks_location": result_of_get_landmark_location,
          "estimated_total_time": 25
        }}

        Respond with ONLY the JSON.
        """

        result = self.determine_route_agent.chat(prompt)
        response_text = str(result.response).strip()

        print("Route agent response received")
        await ctx.set("route_data", response_text)

        try:
            json_match = re.search(r'({[\s\S]*})', response_text)
            if json_match:
                json_str = json_match.group(1)
                route_data = json.loads(json_str)

                await ctx.set("route_data", route_data)

                route_description = f"Walking route planned: Starting from Cohon University Center, visiting {', '.join(route_data['landmarks_to_visit'])}. Total estimated time: {route_data['estimated_total_time']} minutes."

                return TrackingEvent(te_output=route_description)
            else:
                await ctx.set("route_error", "Could not extract JSON from agent response.")
                return TrackingEvent(te_output="Error processing route data")
        except Exception as e:
            await ctx.set("route_error", f"Error processing route data: {str(e)}")
            return TrackingEvent(te_output="Error processing route data")
    @step
    async def TrackRoute(self, ctx: Context, ev: TrackingEvent) -> VerifyPhotoEvent:
        print("Waiting for milestone 1")
        # await asyncio.sleep(0.5)
        print("Waiting for milestone 1")
        # await asyncio.sleep(0.5)
        print("Waiting for milestone 1")
        # await asyncio.sleep(0.5)
        print("Achieved!")
        print("Verify image step and Show Scotty through AR in the location")

        print("Waiting for milestone 2")
        # await asyncio.sleep(0.5)
        print("Waiting for milestone 2")
        # await asyncio.sleep(0.5)
        print("Waiting for milestone 2")
        # await asyncio.sleep(0.5)
        print("Not yet achieved")
        print("Detour, let's make the final path milestone 2, let's go there.")
        print("Milestone 2 reached")
        print("Verify Image Step")

        return VerifyPhotoEvent()

    @step
    async def VerifyPhoto(self, ctx: Context, ev: VerifyPhotoEvent) -> StopEvent | TrackingEvent:
        print("And finally populating DB for Calories Burned and Steps Walked today.")
        return StopEvent(result="Done All Steps")

    @step
    async def FinalStep(self, ev:TrackingEvent) -> StopEvent:
        return StopEvent(result="Done All Steps")


In [155]:
workflow = MyWorkflow(timeout=10, verbose=True)

notification_agent, distance_agent, route_agent, tracking_agent, milestone_agent = create_agents(gemini_api_key)

result = await workflow.run(
    first_input="Start the workflow.",
    when_to_send_notification_agent=notification_agent,
    determine_distance_to_walk_agent=distance_agent,
    determine_route_agent=route_agent,
    track_user_agent=tracking_agent,
    verify_user_milestones_agent=milestone_agent
)
print(result)

Running step setup
Start the workflow.
Step setup produced event BeginProcessingEvent
Running step BeginProcessing
Step BeginProcessing produced event NotificationEvent
Running step CalculateGoalDistance
> Running step 1c5250ee-9989-4175-aacd-530b58346486. Step input: Can you please fetch the distance to walk. Return the distance in miles.
Thought: The current language of the user is: English. I need to use a tool to determine the walking distance.  However, I don't have access to any tools that can directly provide this information.  I need more information, such as a starting point and an ending point.
Answer: I cannot answer the question with the provided tools.  I need more information, such as a starting location and a destination location, to calculate the walking distance.
Step CalculateGoalDistance produced event LandmarksEvent
Running step DetermineRoute
> Running step 571d52da-c6e6-41e4-baa8-7efee81ca5eb. Step input: 
        You are a route planner agent. Use the available t

In [157]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

basic_workflow.html
